In [85]:
import pandas as pd

# Load the dataset
file_path = 'skylab_instagram_datathon_dataset.csv'
data = pd.read_csv(file_path, delimiter=';')

# Clean the data

In [86]:
data.dropna(subset=['followers'], inplace=True)
unique = data["business_entity_doing_business_as_name"].unique()
data['period_end_date'] = pd.to_datetime(data['period_end_date'])
data.set_index('period_end_date', inplace=True)
# interpolate
columns_to_fill = ['likes', 'videos', 'pictures']
# Iterate over each column to fill missing values
for column in columns_to_fill:
    # Iterate over unique companies
    for company in unique:
        # Create a mask for the current company
        company_mask = data["business_entity_doing_business_as_name"] == company
        # Interpolate missing values using time-based method
        data.loc[company_mask, column] = data.loc[company_mask, column].interpolate(method='time')
        # Fill remaining missing values using forward fill and backward fill
        data.loc[company_mask, column] = data.loc[company_mask, column].ffill().bfill()

In [87]:
data

,period,compset_group,compset,business_entity_doing_business_as_name,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,calculation_type,followers,pictures,videos,comments,likes
period_end_date,,,,,,,,,,,,,,
2019-12-07,Weekly,Luxury & Premium & Mainstream,Soft Luxury,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,22066189.0,97.0,13.0,16239.0,6112205.0
2021-01-30,Weekly,Luxury & Premium & Mainstream,Luxury & Premium & Mainstream,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,24289963.0,54.0,2.0,13358.0,3189144.0
2016-05-14,Weekly,Luxury & Premium & Mainstream,Soft Luxury,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,7137442.0,42.0,8.0,15894.0,2152955.0
2019-04-20,Weekly,Luxury & Premium & Mainstream,Luxury & Premium & Mainstream,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,18901125.0,73.0,9.0,26762.0,5224916.0
2023-03-25,Weekly,Luxury & Premium & Mainstream,US Softlines Analyst Interest List,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,30251374.0,54.0,10.0,25681.0,5760987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-21,Weekly,Outdoor Gear,Outdoor Gear,Klean Kanteen,Klean Kanteen Inc,NaN,Klean Kanteen Inc,NaN,Metric Value,242415.0,173.0,4.0,1847.0,27145.0
2018-10-27,Weekly,Outdoor Gear,Outdoor Gear,Klean Kanteen,Klean Kanteen Inc,NaN,Klean Kanteen Inc,NaN,Metric Value,163873.0,125.0,2.0,1764.0,34188.0
2023-08-05,Weekly,Outdoor Gear,Outdoor Gear,Klean Kanteen,Klean Kanteen Inc,NaN,Klean Kanteen Inc,NaN,Metric Value,319485.0,118.0,35.0,1697.0,6328.0


# Load the public companies data

In [88]:
file_path = 'public-companies.csv'
companies = pd.read_csv(file_path, delimiter=',')

In [89]:
companies

,Company Name,Ticker
0,Capri Holdings,CPRI
1,Restaurant Brands International,QSR
2,Pepsi,PEP
3,Monster Energy,MNST
4,Aritzia,ATZAF
...,...,...
202,HanesBrands,HBI
203,Lojas Renner,LREN3.SA
204,Dr. Martens Plc,DOCS.L
205,Yum! Brands,YUM


# Merge the data

In [90]:
data = data.reset_index() 
public_companies = pd.merge(companies, data, left_on='Company Name', right_on='legal_entity_name')
public_companies = public_companies.drop(columns="Company Name")
public_companies = public_companies.rename(columns={"Ticker": "ticker"})

In [91]:
public_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367113 entries, 0 to 367112
Data columns (total 16 columns):
 #   Column                                  Non-Null Count   Dtype         
---  ------                                  --------------   -----         
 0   ticker                                  367113 non-null  object        
 1   period_end_date                         367113 non-null  datetime64[ns]
 2   period                                  367113 non-null  object        
 3   compset_group                           367113 non-null  object        
 4   compset                                 367113 non-null  object        
 5   business_entity_doing_business_as_name  367113 non-null  object        
 6   legal_entity_name                       367113 non-null  object        
 7   domicile_country_name                   356228 non-null  object        
 8   ultimate_parent_legal_entity_name       367113 non-null  object        
 9   primary_exchange_name                

# Load and Merge Yahoo Finance Data

In [92]:
import yfinance as yf

# Function to fetch data from Yahoo Finance
def fetch_yahoo_finance_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    # Fetch historical data within the specified date range and interval
    hist = stock.history(start=start_date, end=end_date, interval="1wk")
    hist.reset_index(inplace=True)
    hist["ticker"] = ticker  # Add a column to identify the ticker
    return hist

tickers = public_companies["ticker"].unique()

finance_data = []

for ticker in tickers:
    dates_of_interest = public_companies[public_companies["ticker"]==ticker]["period_end_date"]
    yahoo_data = fetch_yahoo_finance_data(ticker, min(dates_of_interest), max(dates_of_interest))
    finance_data.append(yahoo_data)

# Concatenate all dataframes into a single dataframe
finance_data = pd.concat(finance_data, ignore_index=True)

finance_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,Capital Gains
0,2015-01-01 00:00:00-05:00,74.320000,74.320000,65.120003,68.709999,28971500,0.0,0.0,CPRI,NaN
1,2015-01-08 00:00:00-05:00,69.099998,70.529999,65.099998,66.519997,17120300,0.0,0.0,CPRI,NaN
2,2015-01-15 00:00:00-05:00,67.360001,69.199997,65.419998,68.500000,13004200,0.0,0.0,CPRI,NaN
3,2015-01-22 00:00:00-05:00,68.779999,71.370003,68.000000,69.989998,13725800,0.0,0.0,CPRI,NaN
4,2015-01-29 00:00:00-05:00,72.470001,73.000000,69.680000,71.379997,18057400,0.0,0.0,CPRI,NaN
...,...,...,...,...,...,...,...,...,...,...
65759,2023-08-14 00:00:00+09:00,4211.702139,4476.157854,4187.215498,4373.313965,1171800,0.0,0.0,8022.T,NaN
65760,2023-08-21 00:00:00+09:00,4348.827535,4627.975248,4290.059595,4471.260742,901500,0.0,0.0,8022.T,NaN
65761,2023-08-28 00:00:00+09:00,4539.823195,4618.180446,4441.876632,4574.104492,466100,0.0,0.0,8022.T,NaN
65762,2023-09-04 00:00:00+09:00,4593.693818,4676.948397,4407.595348,4412.492676,455100,0.0,0.0,8022.T,NaN


In [93]:

min(public_companies[public_companies["ticker"]=="CPRI"]["period_end_date"])

Timestamp('2015-01-03 00:00:00')

In [96]:
finance_data.to_csv("finance_data.csv", index=False)